# MobileNet v2 with CIFAR10

## Librerías

In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torchinfo import summary

import torchvision
from torchvision import models
import torchvision.transforms as transforms

import torchattacks

from utils.evaluation import NormalizationLayer, get_topk_accuracy
from utils.evaluation import plot_adversarial, get_same_predictions, get_different_predictions
from utils.mobilenetv2 import build_mobilenet_v2
from utils.training import train

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device 

device(type='cuda', index=0)

## Modelo
Usaré una versión modificada de MobileNet v2 para trabajar con CIFAR10, ver [PyTorch models trained on CIFAR-10 dataset](https://github.com/huyvnphan/PyTorch_CIFAR10). De hecho ya hay una versión pre-entrenada, pero la versión que usa de PyTorch no es compatible con TorchAttacks, de modo que lo más sencillo es entrenarla de cero. La salida tiene puntuaciones no normalizadas, para obtener probabilidades hay que ejecutar un softmax en la salida.

Por la forma en que funciona [Adversarial-Attacks-PyTorch](https://github.com/Harry24k/adversarial-attacks-pytorch) las imágenes de entrada que se le tienen que pasar deben estar en el rango [0,1], pero los modelos pre-entrenados de PyTorch esperan imágenes normalizadas, las cuáles no están en el [0,1]. La forma de resolver ésto es añadiendo una capa de normalización al inicio. Ver [Demo - White Box Attack (Imagenet)](https://nbviewer.jupyter.org/github/Harry24k/adversarial-attacks-pytorch/blob/master/demos/White%20Box%20Attack%20%28ImageNet%29.ipynb) para un ejemplo con los modelos entrenados en ImageNet.

Lo único que cambia es que las medias y std serán diferentes, ver [How to use models](https://github.com/huyvnphan/PyTorch_CIFAR10#how-to-use-pretrained-models).

In [4]:
# La red entrenada es la que usaré para generar los ejemplos adversarios
mobilenet_v2 = nn.Sequential(
    NormalizationLayer(mean=[0.4914, 0.4822, 0.4465], std=[0.2471, 0.2435, 0.2616]),
    build_mobilenet_v2(pretrained=False))

mobilenet_v2.load_state_dict(torch.load('models/mobilenet_v2.pt'))
mobilenet_v2.eval()

# Lo movemos a la GPU, en caso de que haya
mobilenet_v2 = mobilenet_v2.to(device)

In [5]:
summary(mobilenet_v2)

Layer (type:depth-idx)                             Param #
Sequential                                         --
├─NormalizationLayer: 1-1                          --
├─MobileNetV2: 1-2                                 --
│    └─Sequential: 2-1                             --
│    │    └─ConvBNReLU: 3-1                        928
│    │    └─InvertedResidual: 3-2                  896
│    │    └─InvertedResidual: 3-3                  5,136
│    │    └─InvertedResidual: 3-4                  8,832
│    │    └─InvertedResidual: 3-5                  10,000
│    │    └─InvertedResidual: 3-6                  14,848
│    │    └─InvertedResidual: 3-7                  14,848
│    │    └─InvertedResidual: 3-8                  21,056
│    │    └─InvertedResidual: 3-9                  54,272
│    │    └─InvertedResidual: 3-10                 54,272
│    │    └─InvertedResidual: 3-11                 54,272
│    │    └─InvertedResidual: 3-12                 66,624
│    │    └─InvertedResidual: 3-13   

In [6]:
# Esta red es la que entrenaré
mobilenet_v2_adversarial = nn.Sequential(
    NormalizationLayer(mean=[0.4914, 0.4822, 0.4465], std=[0.2471, 0.2435, 0.2616]),
    build_mobilenet_v2(pretrained=False))

# Lo movemos a la GPU, en caso de que haya
mobilenet_v2_adversarial = mobilenet_v2_adversarial.to(device)

## Dataset & dataloader

In [7]:
transform = transforms.Compose([transforms.ToTensor()])

batch_size = 128

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=4)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=4)

print(f'Trainset: {len(trainset)}')
print(f'Testset: {len(testset)}')

Files already downloaded and verified
Files already downloaded and verified
Trainset: 50000
Testset: 10000


### Adversarial examples

Tomaré 5,000 imágenes de manera aleatoria del conjunto de entrenamiento, posteriormente le aplicaré a cada imagen 4 algoritmos de ataques, por lo que en total tendré al final 20,000 ejemplos adversarios.

In [8]:
indexes = random.sample(range(50000), 5000)
imgs = np.array([trainset.__getitem__(i)[0].numpy() for i in indexes])
labels = np.array([trainset.__getitem__(i)[1] for i in indexes])

len(set(indexes)), np.unique(labels, return_counts=True)

(5000,
 (array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
  array([493, 547, 488, 517, 504, 497, 484, 487, 480, 503])))

In [9]:
imgs_subset = torch.tensor(imgs)
labels_subset = torch.tensor(labels)

trainset_subset = TensorDataset(imgs_subset, labels_subset)
trainloader_subset = DataLoader(trainset_subset, batch_size=32, shuffle=False, num_workers=4)

#### FGSM

In [10]:
%%time

attack = torchattacks.FGSM(mobilenet_v2, eps=1/255)
attack.set_return_type('float') 
attack.save(trainloader_subset, save_path='models/FGSM_train.pt', verbose=True)

- Save Progress: 100.00 % / Accuracy: 47.68 % / L2: 0.21692
- Save Complete!
CPU times: user 7.41 s, sys: 304 ms, total: 7.71 s
Wall time: 6.13 s


#### PGD

In [11]:
%%time

attack = torchattacks.PGD(mobilenet_v2, eps=1/255, alpha=1/255, steps=3)
attack.set_return_type('float') 
attack.save(trainloader_subset, save_path='models/PGD_train.pt', verbose=True)

- Save Progress: 100.00 % / Accuracy: 34.72 % / L2: 0.20776
- Save Complete!
CPU times: user 15.3 s, sys: 325 ms, total: 15.6 s
Wall time: 13.9 s


#### MIFGSM

In [12]:
%%time

attack = torchattacks.MIFGSM(mobilenet_v2, eps=1/255, decay=1.0, steps=3)
attack.set_return_type('float') 
attack.save(trainloader_subset, save_path='models/MIFGSM_train.pt', verbose=True)

- Save Progress: 100.00 % / Accuracy: 39.72 % / L2: 0.20134
- Save Complete!
CPU times: user 15.1 s, sys: 417 ms, total: 15.5 s
Wall time: 13.4 s


#### OnePixel

In [13]:
%%time

attack = torchattacks.OnePixel(mobilenet_v2, pixels=1, steps=5, popsize=20)
attack.set_return_type('float') 
attack.save(trainloader_subset, save_path='models/OnePixel_train.pt', verbose=True)

- Save Progress: 100.00 % / Accuracy: 81.80 % / L2: 0.85973
- Save Complete!
CPU times: user 3min 59s, sys: 470 ms, total: 4min
Wall time: 3min 58s


### Trainset and trainloader

Ahora creamos el conjunto de entrenamiento con los ejemplos adversarios.

In [14]:
images = torch.tensor(np.array([trainset.__getitem__(i)[0].numpy() for i in range(50000)]))
labels = torch.tensor(np.array([trainset.__getitem__(i)[1] for i in range(50000)]))

adv_images_FGSM, adv_labels_FGSM = torch.load('models/FGSM_train.pt')
adv_images_PGD, adv_labels_PGD = torch.load('models/PGD_train.pt')
adv_images_MIFGSM, adv_labels_MIFGSM = torch.load('models/MIFGSM_train.pt')
adv_images_OnePixel, adv_labels_OnePixel = torch.load('models/OnePixel_train.pt')

In [15]:
adversarial_trainset = TensorDataset(torch.cat([images, adv_images_FGSM, adv_images_PGD, adv_images_MIFGSM, adv_images_OnePixel], dim=0),
                                     torch.cat([labels, adv_labels_FGSM, adv_labels_PGD, adv_labels_MIFGSM, adv_labels_OnePixel], dim=0))

adversarial_trainloader = torch.utils.data.DataLoader(adversarial_trainset, batch_size=batch_size, shuffle=True, num_workers=4)

## Entrenamiento

In [16]:
loss_hist, acc_hist = train(mobilenet_v2_adversarial, adversarial_trainloader, testloader, lr=1e-3, epochs=20)

  5%|▌         | 1/20 [01:03<20:00, 63.18s/it]

E00 loss=[ 93.86,106.52] acc=[66.90,61.85]


 10%|█         | 2/20 [02:06<18:56, 63.13s/it]

E01 loss=[ 64.63, 88.68] acc=[77.64,69.41]


 15%|█▌        | 3/20 [03:08<17:49, 62.91s/it]

...

E17 loss=[  6.56, 86.12] acc=[97.75,79.52]


 95%|█████████▌| 19/20 [18:49<00:58, 58.29s/it]

E18 loss=[  6.45, 88.08] acc=[97.83,80.50]


100%|██████████| 20/20 [19:52<00:00, 59.63s/it]

E19 loss=[  6.96, 94.48] acc=[97.56,79.74]


Guardamos el modelo.

In [17]:
torch.save(mobilenet_v2_adversarial.state_dict(), 'models/mobilenet_v2_adversarial.pt')